In [1]:
from qiskit import QuantumCircuit
from qiskit.visualization import plot_bloch_multivector, plot_state_qsphere
from qiskit.quantum_info import Statevector
from qiskit.primitives import StatevectorSampler
from random import getrandbits
import pickle

RETAINED_CHAR = '_'

In [2]:
class CryptoCircuit(QuantumCircuit):
    def __init__(self, size=7):
        # Each qubit representing a bit in ASCII (7-bit)
        super().__init__(size)
        self.sampler = StatevectorSampler()

    def initialize(self, code:str):
        code = code[::-1] # reverse order
        for i in range(len(code)):
            self.x(i) if code[i]!='0' else self.id(i)

    def add_gate(self, gate:bool):
        self.h(self.qubits) if gate else self.id(self.qubits)

    def get_measurements(self):
        self.measure_all()
        result = self.sampler.run([(self)], shots=1).result()[0]
        counts = result.data['meas'].get_counts()
        return list(counts)[0]

    def visualize(self):
        statevector = Statevector(self)
        display(plot_bloch_multivector(statevector))
        # display(plot_state_qsphere(statevector))
    
    def draw(self, output=""):
        return ("You are not allowed to print the circuit..")



In [3]:
class ProtocolBB84:

    def receiver(circuits:list[CryptoCircuit]) -> tuple[list[str], list[int]]:
        letters, gates = [], []
        for circuit in circuits:
            # Skip the already found letters
            if circuit == None:
                letters.append(RETAINED_CHAR)
                gates.append(None)
                continue

            # Choose a random receiving basis
            gates.append(getrandbits(1))
            circuit.add_gate(gates[-1])

            # Measure the qubits to get the received binary code
            letter = circuit.get_measurements()
            # Get letter from binary code
            letters.append(chr(int(letter,2)))

        return letters, gates


    def classicalChannel(sender_gates, receiver_gates):
        correct_chars = []
        for i, (sent,received) in enumerate(zip(sender_gates, receiver_gates)):
            if sent == received :
                correct_chars.append(i)
        return correct_chars



In [4]:

# Receive - read from file
with open("quantum_channel.txt", "br") as f:
    circuits_in = f.read().split(b'\n\n')[:-1]

with open("classical_channel.txt", "r") as f:
    sender_gates = [int(gate) for gate in f.read()]


# In real example this loop would not be possible since the quantum state would colapse
final_message = RETAINED_CHAR *len(sender_gates)
while RETAINED_CHAR in final_message:
    received_message, receiver_gates = ProtocolBB84.receiver([pickle.loads(circuit) for circuit in circuits_in])

    correct_chars = ProtocolBB84.classicalChannel(sender_gates, receiver_gates)
    received_correctly = ''.join([letter if i in correct_chars else RETAINED_CHAR for i,letter in enumerate(received_message)])
    
    final_message = ''.join([ letter if letter!=RETAINED_CHAR else received_correctly[i] for i, letter in enumerate(final_message) ])

    print(received_correctly, '->', final_message)



B_84 _s _ _u___um_K_y ___t___ut___ _r_toc___re__ing __ t_e_n___loni_____eo__m. -> B_84 _s _ _u___um_K_y ___t___ut___ _r_toc___re__ing __ t_e_n___loni_____eo__m.
BB84 is_a Qu__tu_ _ey_D_s_rib____n __oto_______ying on_the_n_-c____ng _he_r___ -> BB84 is a Qu__tum Key D_stribut__n _rotoc___re_ying on the_n_-cloning _heor_m.
_B8_ __ a_Qua_tum__e___i__ri___io_ pr_t__o___e____g _n _he__o_c__n____th__rem_ -> BB84 is a Qua_tum Key Distribution protoco__re_ying on the_no-cloning theorem.
__84 __ a__ua__u_ K__ _ist_ibuti_n pro___o__r_ly_ng_on __e __-_l_n______eo_em. -> BB84 is a Qua_tum Key Distribution protoco__relying on the no-cloning theorem.
B____i_ __Q__ntu_ ___________u__on __ot____ r_ly_ng o_ ____n_____n__g _h_ore_. -> BB84 is a Quantum Key Distribution protoco_ relying on the no-cloning theorem.
___4_____ _ua_t_m__e_ _istr_but__n______co_ re____g___ _h____-cl_____ th_o__m_ -> BB84 is a Quantum Key Distribution protoco_ relying on the no-cloning theorem.
__84 is_a___an_u_ ___ _i_t__buti__